## Задача 4-1. Линейное программирование: формулирование задачи.

В этой задаче Вам предлагается закодировать задачу TSP для заданного графа в виде задачи ЦЛП с помощью условий MTZ (Миллера—Таккера—Землина) и поработать с библиотекой [PuLP](https://pypi.python.org/pypi/PuLP/1.6.5). Если Вы используете дистрибутив Anaconda, то эта библиотека не находится утилитой conda, зато совершенно нормально устанавливается с помощью pip:

`pip install pulp`

Вам дана функция `dist15`, которая по двум номерам вершин возвращает вес ребра между ними. Номера от 1 до 15, пример взят [отсюда](https://people.sc.fsu.edu/~jburkardt/datasets/tsp/tsp.html).
Нужно построить соответствующую задачу ЦЛП и решить её средствами PuLP, чтобы найти оптимальный гамильтонов цикл в графе. Сделать это следует в функции `solve_tsp_with_lp`, которая получает на вход размерность задачи и весовую функцию, а на выходе даёт перестановку номеров вершин графа (нумеруемых с единицы), соответствующую оптимальному гамильтонову циклу.

** Импортируем нужные модули: **

In [51]:
import itertools

from pulp import LpVariable, LpProblem, LpConstraint, LpMinimize, LpAffineExpression, LpInteger
from typing import List, Callable

** Функция, возвращающая вес ребра между данными вершинами: **

In [52]:
def dist15(i: int, j: int) -> int:
    return (
        list(
            map(
                int, 
                filter(
                    lambda s: len(s.strip()) > 0, '''
                    0 29 82 46 68 52 72 42 51 55 29 74 23 72 46 
                    29  0 55 46 42 43 43 23 23 31 41 51 11 52 21 
                    82 55  0 68 46 55 23 43 41 29 79 21 64 31 51 
                    46 46 68  0 82 15 72 31 62 42 21 51 51 43 64 
                    68 42 46 82  0 74 23 52 21 46 82 58 46 65 23 
                    52 43 55 15 74  0 61 23 55 31 33 37 51 29 59 
                    72 43 23 72 23 61  0 42 23 31 77 37 51 46 33 
                    42 23 43 31 52 23 42  0 33 15 37 33 33 31 37 
                    51 23 41 62 21 55 23 33  0 29 62 46 29 51 11 
                    55 31 29 42 46 31 31 15 29  0 51 21 41 23 37 
                    29 41 79 21 82 33 77 37 62 51  0 65 42 59 61 
                    74 51 21 51 58 37 37 33 46 21 65  0 61 11 55 
                    23 11 64 51 46 51 51 33 29 41 42 61  0 62 23 
                    72 52 31 43 65 29 46 31 51 23 59 11 62  0 59 
                    46 21 51 64 23 59 33 37 11 37 61 55 23 59  0
                    '''.split()
                )
            )
        )[(i-1) * 15 + (j-1)]
    )

** Вычисление веса цикла по списку его вершин: **

In [53]:
def cycle_weight(vertices):
    size = len(vertices)
    weight = dist15(vertices[size - 1], vertices[0])
    for index, vertex in enumerate(vertices):
        if (index < size - 1):
            weight += dist15(vertices[index], vertices[index + 1])
    return weight

** Получаем лист: **

In [54]:
into_chain = lambda l: list(itertools.chain.from_iterable(l))

** Решение основной задачи: **

In [59]:
def solve_tsp_with_lp(num_vertices: int, distance_function: Callable[[int, int], int]) -> List[int]:
    
    # Удобоиспользуемые переменные:
    V = num_vertices
    E = 15 * 14 / 2
    
    # Инициализация решаемой задачи:
    TSP = LpProblem(name="TSP_MTZ", sense=LpMinimize)
    
    # Добавляем переменные u_{i} (MTZ-переменные):
    MTZ_variables = [LpVariable(name='u_{}'.format(i), lowBound=1, upBound=V, cat='Integer') for i in range(V)]
    
    # Добавляем наши n^2 переменных (веса ребер):
    weight_variables = [
        [LpVariable(name="weight_{}_{}".format(i, j), lowBound = 0, upBound = 1, cat = 'Integer') for j in range(V)]
                        for i in range(V)]
    
    # Добавляем субъект оптимизации:
    TSP.objective = LpAffineExpression( into_chain([
                [(weight_variables[i][j], distance_function(i + 1, j + 1)) for j in range(V) if i != j]
                                                 for i in range(V)]) )
    
    # Ограничиваем число входящих в вершину ребер и исходящих из нее единицей (мы ведь ищем цикл):
    for vertex_num in range(V):
        TSP += (LpAffineExpression([(weight_variables[i][vertex_num], 1) 
                                for i in range(V) if i != vertex_num]) == 1)
        TSP += (LpAffineExpression([(weight_variables[vertex_num][i], 1)
                                for i in range(V) if i != vertex_num]) == 1)
       
    # Добавляем ограничения MTZ (u_{i} - u_{j} + V * weight_variables_{ij} <= V - 1)
    for i in range(V):
        for j in range(1, V):
            if i != j:
                TSP += (MTZ_variables[i] - MTZ_variables[j] + V * weight_variables[i][j] <= V - 1)
    
    # Решаем задачу, возвращаем ответ:
    TSP.solve()
    answer = [1]
    current = 0
    while len(answer) < V:
        for vertex_number in range(V):
            if weight_variables[current][vertex_number].value() == 1.0:
                answer.append(vertex_number + 1)
                current = vertex_number
                
    return answer

** Выводим решение основной задачи: **

In [61]:
answer = solve_tsp_with_lp(15, dist15)
print("Вес найденного ответа: {}".format(cycle_weight(answer)))
print("Найденный цикл минимального веса: {}".format(' ⟶ '.join(map(str, answer + [1]))))

Вес найденного ответа: 291
Найденный цикл минимального веса: 1 ⟶ 13 ⟶ 2 ⟶ 15 ⟶ 9 ⟶ 5 ⟶ 7 ⟶ 3 ⟶ 12 ⟶ 14 ⟶ 10 ⟶ 8 ⟶ 6 ⟶ 4 ⟶ 11 ⟶ 1


** Вывод: **

Благодаря этой работе, мы не только научились пользоваться замечательной библиотекой PuLP, но и решили задачу TSP в виде ЦЛП с помощью условий Миллера-Таккера-Землина!